# Clean Slate: Estimating offenses eligible for expungement under varying conditions
> Prepared by [Laura Feeney](https://github.com/laurafeeney) for Code for Boston's [Clean Slate project](https://github.com/codeforboston/clean-slate).

## Purpose
The purpose of this notebook is to get a list of all charges from both Northwestern and Suffolk data, then map expungement eligibility to each.

The resulting datasets will be output as a separate .csv that can be save and shared in the Clean Slate GitHub repo.


The purpose of this notebook is to join prosecution charges with expungability, and estimate the number of offenses which are eligible for expungement. This analysis will be run a number of times: under existing regulation, and under varying conditions allowing additional offenses to be expunged.

The target questions, provided June 28, 2020: 

1. How many people (under age 21) are eligible for expungement today? This would be people with only **one charge** that is not part of the list of ineligible offenses (per section 100J). 


2. How many people (under age 21) would be eligible based on only having **one incident** (which could include multiple charges) that are not part of the list of ineligible offenses?
 - How many people (under age 21) would be eligible based on only having **one incident** if only sex-based offenses or murder were excluded from expungement?
 

3. How many people (under age 21) would be eligible based on who has **not been found guilty** (given current offenses that are eligible for expungement)?
 - How many people (under age 21) would be eligible based on who has **not been found guilty** for all offenses except for murder or sex-based offenses?

-----

### Step 0
Import data, programs, etc.

-----

In [1]:
import pandas as pd
pd.set_option("display.max_rows", 200)
import numpy as np
import regex as re
import glob, os
import datetime 
from datetime import date 

In [2]:
# processed individual-level data from NW district with expungability.

nw = pd.read_csv('../../data/processed/merged_nw.csv', encoding='cp1252',
                    dtype={'Analysis notes':str, 'extra_criteria':str, 'Expungeable': str}) 
#nw.head()
nw['Expungeable'].value_counts(dropna=False)

#nw.loc[nw['Person ID']=="NW-12460", ['CMRoffense']] = "no"

Yes              55052
No               20013
NotApplicable      439
Attempt            220
NaN                  1
Name: Expungeable, dtype: int64

## Step 1
Add a few additional variables needed, look at summary stats

There are many offenses that are violations of the Code of Massachusetts Regulations (CMR) rather than a criminal offense. These include things like some driving or boating infractions (e.g., not having headlights on), or not having a hunting/fishing license. It's not clear whether these should be included at all. We'll run the analysis first including them at all stages, and then excluding them at all stages. 

In [3]:
### number of unique people
nw['Person ID'].nunique()

19753

In [4]:
### dates ###

reference_date = datetime.date(2020, 9, 1) # using "today.date() wouldn't be stable"

nw['Offense Date'] = pd.to_datetime(nw['Offense Date']).dt.date
nw['years_since_offense'] = (reference_date - nw['Offense Date'])/pd.Timedelta(1, 'D')/365

nw[nw['Offense Date']<datetime.date(1950, 1, 1)]

print("The earliest offense date is", min(nw['Offense Date']))
print("The max offense date is", max(nw['Offense Date']), "\n")

print(nw['years_since_offense'].describe())

print("\n There are a tail of dates that are probably wrong, but age is missing for most, and the most egregious:")
nw[nw['years_since_offense'] > nw['years_since_offense'].quantile(.9997)][[
    'Offense Date', 'Age at Offense', 'years_since_offense']]

The earliest offense date is 1750-12-03
The max offense date is 2018-12-30 

count    74915.000000
mean         4.460396
std          2.798864
min          1.673973
25%          3.060274
50%          4.328767
75%          5.638356
max        269.926027
Name: years_since_offense, dtype: float64

 There are a tail of dates that are probably wrong, but age is missing for most, and the most egregious:


,Offense Date,Age at Offense,years_since_offense
2293,1882-11-30,NaN,137.846575
2294,1882-11-30,NaN,137.846575
2295,1882-11-30,NaN,137.846575
43250,1975-06-30,NaN,45.205479
54403,1961-05-14,NaN,59.342466
56406,1750-12-03,NaN,269.926027
56407,1750-12-03,NaN,269.926027
60782,1975-08-01,28.0,45.117808
60783,1975-08-01,28.0,45.117808
60784,1975-08-01,28.0,45.117808


In [5]:
# distribution of # of charges

nw['num_offenses']=nw.groupby('Person ID')['Person ID'].transform('count')
print(nw['num_offenses'].describe())

#nw.loc[nw['num_offenses']<30].hist(column='num_offenses', bins=15)

count    75725.000000
mean        10.072539
std         18.186474
min          1.000000
25%          3.000000
50%          5.000000
75%         11.000000
max        202.000000
Name: num_offenses, dtype: float64


In [6]:
nw['num_offenses_noCMR']=nw.groupby(['Person ID', 'CMRoffense'])['Person ID'].transform('count')
nw.loc[nw['CMRoffense']=="yes", ['num_offenses_noCMR']] = None
nw["num_offenses_noCMR"] = nw.groupby("Person ID")["num_offenses_noCMR"].transform(lambda x: x.fillna(x.mean()))
nw['num_offenses_noCMR']= nw['num_offenses_noCMR'].fillna(0)

nw['num_CMR']=nw.groupby(['Person ID', 'CMRoffense'])['Person ID'].transform('count')
nw.loc[nw['CMRoffense']=="no", ['num_CMR']] = None
nw["num_CMR"] = nw.groupby("Person ID")["num_CMR"].transform(lambda x: x.fillna(x.mean()))
nw['num_CMR'] = nw['num_CMR'].fillna(0)\


In [7]:
a = nw[nw['num_offenses']==1]['Person ID'].nunique()

b = nw[nw['num_offenses_noCMR']==1]['Person ID'].nunique()

#people only only ever had 1 offense, which was not a CMR
c = nw.loc[
    (nw['num_CMR'] ==0) & 
    (nw['num_offenses']==1)][
    'Person ID'].nunique()

#people newly included as eligible, because CMRs were adding up to count against them as other offenses
d = nw.loc[
    (nw['num_offenses'] - nw['num_CMR']    ==1) & 
    (nw['num_offenses'] > 1)][
    'Person ID'].nunique()

# people newly excluded from the eligible count, because they had only 1 offense, and it was a CMR
e = nw.loc[
    (nw['num_CMR']==1) & 
    (nw['num_offenses']==1)][
    'Person ID'].nunique()

#print(a, b, c, d, e)
print("# People with only 1 offense, incl CMR: ", a)
print("# People with only 1 offense, excluding CMR from multiple and indvl counts:", b)
print("It is strange that these numbers are the same, but it's just a coincidence.")
print("# people with only have 1 offense and no CMRs (unaffected by excluding CMRs) is: ", c)
print("# people newly included in # with only 1 offense if CMRs don't count against them is: ", d)
print("and # people with only 1 offense, which was a CMR and so aren't in the new count is also:", e, '\n')

x = nw.drop_duplicates(subset=['Person ID'])
pd.crosstab(x.loc[x['num_offenses']<11]['num_offenses_noCMR'], 
            x.loc[x['num_offenses']<11]['num_offenses'], margins = True, dropna = False)

# People with only 1 offense, incl CMR:  5026
# People with only 1 offense, excluding CMR from multiple and indvl counts: 5026
It is strange that these numbers are the same, but it's just a coincidence.
# people with only have 1 offense and no CMRs (unaffected by excluding CMRs) is:  4963
# people newly included in # with only 1 offense if CMRs don't count against them is:  63
and # people with only 1 offense, which was a CMR and so aren't in the new count is also: 63 



num_offenses,1,2,3,4,5,6,7,8,9,10,All
num_offenses_noCMR,,,,,,,,,,,
0.0,63,4,0,0,0,0,0,0,0,0,67
1.0,4963,56,7,0,0,0,0,0,0,0,5026
2.0,0,4790,73,3,0,0,0,0,0,0,4866
3.0,0,0,3276,39,4,0,0,0,0,0,3319
4.0,0,0,0,1729,33,2,0,0,0,0,1764
5.0,0,0,0,0,1175,26,2,0,0,0,1203
6.0,0,0,0,0,0,814,21,0,0,0,835
7.0,0,0,0,0,0,0,531,15,0,0,546
8.0,0,0,0,0,0,0,0,401,12,1,414


In [8]:
# snapshot of some offenses people have committed where they have both CMR and non-CMR

nw[['Person ID', 'num_offenses','num_offenses_noCMR', 'Charge']].loc[
    (nw['num_offenses']!=nw['num_offenses_noCMR']) &
    (nw['num_CMR']>0)
    ]['Charge'].value_counts(normalize=True)

STATE HWAYÃ‚â€”TRAFFIC VIOLATION * 720 CMR Ã‚Â§9.06               0.081253
LICENSE SUSPENDED, OP MV WITH c90 Ã‚Â§23                          0.061734
UNLICENSED OPERATION OF MV c90 Ã‚Â§10                             0.050386
NEGLIGENT OPERATION OF MOTOR VEHICLE c90 Ã‚Â§24(2)(a)             0.048116
MARKED LANES VIOLATION * c89 Ã‚Â§4A                               0.039492
                                                                    ...   
DEER TAGGING VIOLATION c131 Ã‚Â§72                                0.000454
DRUG, POSSESS TO DISTRIB CLASS A, SUBSQ. c94C Ã‚Â§32(b)           0.000454
UNINSURED MOTOR VEHICLE, SUBSQ.OFF                                0.000454
COCAINE, POSSESS TO DISTRIBUTE c94C Ã‚Â§32A(c)                    0.000454
LEAVE SCENE OF PERSONAL INJURY & DEATH c90 Ã‚Â§24(2)(aÃ‚Â½)(2)    0.000454
Name: Charge, Length: 240, dtype: float64

In [9]:
print(nw['Expungeable'].value_counts())
pd.crosstab(nw['Expungeable'], nw['CMRoffense'], dropna=False)

Yes              55052
No               20013
NotApplicable      439
Attempt            220
Name: Expungeable, dtype: int64


CMRoffense,no,yes
Expungeable,,
Attempt,220,0
No,20013,0
NotApplicable,0,439
Yes,55052,0


## Question 1
- How many people are eligible for expungement today? 
    - ---> *Only one charge*, Offense committed before 21st birthday, charge is not part of the list of ineligible offenses (per section 100J), charge is in the correct timeframe from today's date

We don't have misdemeanor / felony info, so will show the # that are more than 3 or more than 7 years from today.

----

In [10]:
eligible_all_ages = nw.loc[
    (nw['num_offenses']==1) &
    (nw['Expungeable'] != 'No') &
    (~nw['Age at Offense'].isnull())
]['Person ID'].nunique()


eligible_all_ages_noCMR = nw.loc[
    (nw['num_offenses_noCMR']==1) &
    (nw['Expungeable'] != 'No') &
    (~nw['Age at Offense'].isnull()) &
    (nw['CMRoffense'] == "no") # exclude rows where a CMR offense committed <21 might include them in this list
]['Person ID'].nunique()

print("Before filtering on age, there are", eligible_all_ages, "eligible w/ CMR", "and", eligible_all_ages_noCMR, "wo CMR.")

Before filtering on age, there are 3546 eligible w/ CMR and 3533 wo CMR.


In [11]:

x = nw.loc[
    (nw['num_offenses']==1) &
    (nw['Expungeable'] != 'No') &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21)
]

People_eligible = x['Person ID'].nunique()

greater3 = x.loc[
    (x['years_since_offense'] > 3)
]['Person ID'].nunique()

greater7 = x.loc[
    (x['years_since_offense'] > 7)
]['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if we consider CMR to be crimes,",
      "and we include 'attempts' as expungeable.")
print("Of these,", greater3, "occured more than 3 years before", reference_date)
print("Of these,", greater7, "occured more than 7 years before", reference_date)

There are 535 people with charges eligible for expungement in the NW district of MA, if we consider CMR to be crimes, and we include 'attempts' as expungeable.
Of these, 384 occured more than 3 years before 2020-09-01
Of these, 11 occured more than 7 years before 2020-09-01


In [12]:
x = nw.loc[
    (nw['num_offenses_noCMR']==1) &
    (nw['Expungeable'] != 'No') &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) &
    (nw['CMRoffense'] == "no") # exclude rows where a CMR offense committed <21 might include them in this list
]

People_eligible = x['Person ID'].nunique()

greater3 = x.loc[
    (x['years_since_offense'] > 3)
]['Person ID'].nunique()

greater7 = x.loc[
    (x['years_since_offense'] > 7)
]['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if we ignore CMR violations",
     "and consider attempts to be expungeable.")
print("Of these,", greater3, "occured more than 3 years before", reference_date)
print("Of these,", greater7, "occured more than 7 years before", reference_date)

There are 525 people with charges eligible for expungement in the NW district of MA, if we ignore CMR violations and consider attempts to be expungeable.
Of these, 373 occured more than 3 years before 2020-09-01
Of these, 10 occured more than 7 years before 2020-09-01


In [13]:

x = nw.loc[
    (nw['num_offenses']==1) &
    (nw['Expungeable'] == 'Yes') &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

greater3 = x.loc[
    (x['years_since_offense'] > 3)
]['Person ID'].nunique()

greater7 = x.loc[
    (x['years_since_offense'] > 7)
]['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if we consider CMR to be crimes,",
      "and we assume all 'attempts' are not expungeable.")
print("Of these,", greater3, "occured more than 3 years before", reference_date)
print("Of these,", greater7, "occured more than 7 years before", reference_date)

There are 521 people with charges eligible for expungement in the NW district of MA, if we consider CMR to be crimes, and we assume all 'attempts' are not expungeable.
Of these, 370 occured more than 3 years before 2020-09-01
Of these, 10 occured more than 7 years before 2020-09-01


In [14]:

x = nw.loc[
    (nw['num_offenses_noCMR']==1) &
    (nw['Expungeable'] == 'Yes') &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) &
    (nw['CMRoffense'] == "no") # exclude rows where a CMR offense committed <21 might include them in this list
]

People_eligible = x['Person ID'].nunique()

greater3 = x.loc[
    (x['years_since_offense'] > 3)
]['Person ID'].nunique()

greater7 = x.loc[
    (x['years_since_offense'] > 7)
]['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if we ignore CMR violations",
     "and assume attempts are not expungeable.")
print("Of these,", greater3, "occured more than 3 years before", reference_date)
print("Of these,", greater7, "occured more than 7 years before", reference_date)

There are 523 people with charges eligible for expungement in the NW district of MA, if we ignore CMR violations and assume attempts are not expungeable.
Of these, 371 occured more than 3 years before 2020-09-01
Of these, 10 occured more than 7 years before 2020-09-01


## Question 2: incidents rather than offenses

Luke Schissler

**Question 2a** - If we only change the number of offenses, how many are eligible based on only having one incident (that does not include an ineligible offense)? 

Making the assumption based on the 06/26/20 meeting notes that an 'one incident' is defined as the sum of all charges for an individual on a single day. In theory, someone could probably be charged in two separate instances on a single day, but this is likely rare. 

Another question is whether a single non-expungeable offense under the bundle of the incidence makes the entire incidence non-expungeable. For this analysis, I am proceeding with the assumption that one non-expungeable incident in the batch makes the entire thing non-expungeable.

In [120]:
incidents = nw.groupby('Person ID')['Offense Date'].nunique()

def generate_incident_hash(dic):
    for index, row in nw.iterrows():
        if row['Expungeable'] == 'No':
            dic[(row['Person ID'], row["Offense Date"])] = 'No'
    return dic

def generate_murder_sex_hash(dic):
    for index, row in nw.iterrows():
        if row['murder'] == 1 or row['sex'] == 1:
            dic[(row['Person ID'], row['Offense Date'])] = 1
    return dic

incident_dict = generate_incident_hash({})
murder_sex_dict = generate_murder_sex_hash({})

def find_inc(per_id):
    return incidents[per_id]

nw['Incidents'] = np.vectorize(find_inc)(nw['Person ID']) #Incidents column is number of unique offense dates per Person
nw['Incident Expungeable'] = np.vectorize(lambda x, y : incident_dict.get((x, y), 'Yes'))(nw['Person ID'], nw['Offense Date'])
nw['Incident Murder/Sex'] = np.vectorize(lambda x, y : murder_sex_dict.get((x, y), 0))(nw['Person ID'], nw['Offense Date'])

In [112]:
x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['Incident Expungeable'] == 'Yes') &
    (nw['Expungeable'] == 'Yes') &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

greater3 = x.loc[
    (x['years_since_offense'] > 3)
]['Person ID'].nunique()

greater7 = x.loc[
    (x['years_since_offense'] > 7)
]['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has a single incident, in which all the offenses are expungeable.")
print("Of these,", greater3, "occured more than 3 years before", reference_date)
print("Of these,", greater7, "occured more than 7 years before", reference_date)

print("\n")
print(x.sample(n=3)) # grab a random sample to verify

There are 1195 people with charges eligible for expungement in the NW district of MA, if the person has a single incident, in which all the offenses are expungeable.
Of these, 829 occured more than 3 years before 2020-09-01
Of these, 21 occured more than 7 years before 2020-09-01


      Person ID Offense Date  Age at Offense       Filed  Status  Count  \
67832  NW-16675   2018-01-26            20.0  2018-01-29  Closed      2   
70274  NW-17591   2017-12-22            19.0  2018-05-15  Closed      3   
10926   NW-1958   2014-03-17            20.0  2014-04-15  Closed      4   

                                         Charge            Disposition  \
67832  MISCELLANEOUS MUNIC ORDINANCE/BYLAW VIOL        c276s87 finding   
70274       INSPECTION/STICKER, NO * c90 Ã‚Â§20        Not Responsible   
10926   CREDIT CARD, LARCENY OF c266 Ã‚Â§37B(b)  Continued w/o Finding   

       Dispo Date CMRoffense  ... extra_criteria  \
67832  2018-01-29         no  ...            NaN   
70274  2018-05-

**Question 2b** -  How many would be eligible if we only limited to only exclude sex-based offenses or murder?

Assuming here we are not limiting based on expungability, but only on if the person has 1 incident, it is related to neither murder nor sex, and the person is under 21. 

In [122]:
x = nw.loc[
    (nw['Incidents'] == 1) &
    (nw['Incident Murder/Sex'] == 0) &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

greater3 = x.loc[
    (x['years_since_offense'] > 3)
]['Person ID'].nunique()

greater7 = x.loc[
    (x['years_since_offense'] > 7)
]['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has a single incident in which all offenses are unrelated to sex or murder.")
print("Of these,", greater3, "occured more than 3 years before", reference_date)
print("Of these,", greater7, "occured more than 7 years before", reference_date)

print("\n")
print(x.sample(n=3)) # grab a random sample to verify

There are 1715 people with charges eligible for expungement in the NW district of MA, if the person has a single incident in which all offenses are unrelated to sex or murder.
Of these, 1215 occured more than 3 years before 2020-09-01
Of these, 30 occured more than 7 years before 2020-09-01


      Person ID Offense Date  Age at Offense       Filed  Status  Count  \
51794  NW-11427   2015-12-11            12.0  2016-07-12  Closed      1   
2127     NW-410   2013-11-28            20.0  2014-01-22  Closed      2   
51711  NW-11409   2016-05-12            20.0  2016-07-11  Closed      2   

                                  Charge            Disposition  Dispo Date  \
51794                   A&B c265 Ã‚Â§13A  Continued w/o Finding  2016-12-13   
2127               SPEEDING * c90 Ã‚Â§17            Responsible  2014-02-06   
51711  THREAT TO COMMIT CRIME c275 Ã‚Â§2  Continued w/o Finding  2016-10-12   

      CMRoffense  ... Reason not expungeable Analysis notes  Expungeable  \
51794       

## Question 3: Verdict as Determiner

**Question 3a** -  How many would be eligible based on who has not been found guilty (given current offenses that are eligible for expungement)?

In [47]:
x = nw.loc[
    (nw['Expungeable'] == 'Yes') &
    (~nw['Disposition'].isnull()) &
    (nw['Disposition'] != 'Guilty') &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

greater3 = x.loc[
    (x['years_since_offense'] > 3)
]['Person ID'].nunique()

greater7 = x.loc[
    (x['years_since_offense'] > 7)
]['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has an expungeable offense and has a verdict other than guilty.")
print("Of these,", greater3, "occured more than 3 years before", reference_date)
print("Of these,", greater7, "occured more than 7 years before", reference_date)

print("\n")
print(x.sample(n=3)) # grab a random sample to verify

There are 2335 people with charges eligible for expungement in the NW district of MA, if the person has an expungable offense and has a verdict other than guilty.
Of these, 1823 occured more than 3 years before 2020-09-01
Of these, 38 occured more than 7 years before 2020-09-01
      Person ID Offense Date  Age at Offense       Filed  Status  Count  \
55348  NW-12538   2016-10-29            19.0  2016-10-31  Closed      1   
43848   NW-9166   2016-05-20            15.0  2016-07-01  Closed      1   
24210   NW-4498   2014-08-17            19.0  2014-10-08  Closed      2   

                                             Charge  \
55348  LIQUOR, PERSON UNDER 21 POSSESS c138 Ã‚Â§34C   
43848                DISORDERLY CONDUCT c272 Ã‚Â§53   
24210           UNINSURED MOTOR VEHICLE c90 Ã‚Â§34J   

                        Disposition  Dispo Date CMRoffense  ... murder  \
55348               c276s87 finding  2016-10-31         no  ...      0   
43848                Nolle Prosequi  2017-01-20    

**Question 3b** -  How many would be eligible based on who has not been found guilty, except murder or sex offenses?


In [53]:
x = nw.loc[
    (nw['murder'] == 0) &
    (nw['sex'] == 0) &
    (~nw['Disposition'].isnull()) &
    (nw['Disposition'] != 'Guilty') &
    (~nw['Age at Offense'].isnull()) &
    (nw['Age at Offense']<21) 
]

People_eligible = x['Person ID'].nunique()

greater3 = x.loc[
    (x['years_since_offense'] > 3)
]['Person ID'].nunique()

greater7 = x.loc[
    (x['years_since_offense'] > 7)
]['Person ID'].nunique()

print("There are", People_eligible,
      "people with charges eligible for expungement in the NW district of MA, if the person has an offense unrelated to sex or murder and has a verdict other than guilty.")
print("Of these,", greater3, "occured more than 3 years before", reference_date)
print("Of these,", greater7, "occured more than 7 years before", reference_date)

print("\n")
#print(x.sample(n=3)) # grab a random sample to verify

There are 2668 people with charges eligible for expungement in the NW district of MA, if the person has an offense unrelated to sex or murder and has a verdict other than guilty.
Of these, 2125 occured more than 3 years before 2020-09-01
Of these, 56 occured more than 7 years before 2020-09-01


      Person ID Offense Date  Age at Offense       Filed  Status  Count  \
10780   NW-1921   2014-04-11            19.0  2014-04-14  Closed      1   
32646   NW-6400   2015-02-23            17.0  2015-03-18  Closed     10   
36911   NW-7395   2015-05-17            16.0  2015-06-15  Closed      1   

                                             Charge  \
10780  LIQUOR, PERSON UNDER 21 POSSESS c138 Ã‚Â§34C   
32646             LARCENY UNDER $250 c266 Ã‚Â§30(1)   
36911         UNLICENSED OPERATION OF MV c90 Ã‚Â§10   

                        Disposition  Dispo Date CMRoffense  ... murder  \
10780               c276s87 finding  2014-05-22         no  ...      0   
32646         Continued w/o Findi